In [9]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "Casper32"
host = 'nv-desktop-services.apporto.com'
port = 30548
db = 'AAC'
collection = 'animals'

#db = AnimalShelter(username, password)
shelter = AnimalShelter()


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'], inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image

if not os.path.isfile(image_filename):
    raise FileNotError(f"The file at {image_filename} does not exist or is a directory")

with open(image_filename, 'rb') as image_file:
    encoded_image = base64.b64encode(image_file,read()).decode()

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Latoria CS-340 Dashboard'))),
    html.Hr(),
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    html.Div([
        ddc.Dropdown(
            id='filter-type',
            options=[
                {'label':'Water Rescue', 'value': 'Water Rescue'},
                {'label':'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label':'Disaster or Individual Tracking', 'value':'Disaster or Individual Tracking'},
                {'label': 'All', 'value': 'All'},

            ],
            value='All'  
        ),
        html.Img(src=f'data:image/png;base64,{encoded_image}', style={'width': '200px'})
    ]),
    
    html.Hr(),
    
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        row_selectable="single", #allow a row to be selected
        selected_rows=[0],
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left'},
        page_size=10, #set rows per page
        style_header={
            'backgroundColor': 'rgb(230,230,230)',
            'fontWeight': 'bold'
        },
        style_data={
            'whiteSpace': 'normal',
            'height': 'auto'
        }
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 

    ),
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row', style={'display' : 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='graph-id2', className='col s12 m6')
    ]),
    html.Div(id='map-id', className='col s12', style={'width': '100%', 'height': '500px'})
])
                       
app.config.suppress_callback_exceptions = True

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(
    [Output('datatable-id','data'),
    Output('graph-id','children'),
    Output('graph-id2', 'children')],
    [Input('filter-type', 'value')]
)
def update_table_and_charts(filter_type):
## FIX ME Add code to filter interactive data table with MongoDB queries
#
    if filter_type == 'Water Rescue':
        filtered_df = df[df['breed'].str.contains('Labrador Retriever|Newfoundland', case=False, na=False)]
    elif filter_type == 'Mountain or Wilderness Rescue':
        filtered_df = df[df['breed'].str.contains('German Shepherd|Bernese Mountain Dog', case=False, na=False)]
    elif filter_type == 'Diaster or Individual Tracking':
        filtered_df = df[df['breed'].str.contains('Bloodhound|Beagle', case=False, na=False)]
    else:
        filtered_df = df
    
    table_data = filtered_df.to_dict('records')
    bar_fig = px.bar(filtered_df, x='animal_type', title="Count of Animal Types")
    
    pie_fig = px.pie(filtered_df, names='outcome_type', title="Distribution of Outcome Types")                
    pie_chart = dcc.Graph(figure=bar_fig)
                       
    return table_data, bar_chart, pie_chart
# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_map(viewData, index):
    dff = pd.DataFrame.from_dict(viewData)
    
    if index is None or len(index) == 0:
        row = 0
    else:
        row = index[0]
    
    lat = dff.iloc[row, dff.columns.get_loc('location_lat')] if pd.notna(dff.iloc[row, dff.columns.get_loc('location_lat')]) else 30.75
    lon = dff.iloc[row, dff.columns.get_loc('location_long')] if pd.notna(dff.iloc[row, ddf.columns.get_loc('location_long')]) else -97.48

    return [
        dl.Map(style={'width': '100%', 'height': '500px'}, 
               center=[lat, lon], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(dff.iloc[row, dff.columns.get_loc('breed')]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row, dff.columns.get_loc('name')])
                ])
            ])
        ])
    ]

app.run_server(debug=True)

ServerSelectionTimeoutError: nv-desktop-services.apporto.com:30548: [Errno 111] Connection refused, Timeout: 30s, Topology Description: <TopologyDescription id: 66cbd67020d6d9fe812344ab, topology_type: Single, servers: [<ServerDescription ('nv-desktop-services.apporto.com', 30548) server_type: Unknown, rtt: None, error=AutoReconnect('nv-desktop-services.apporto.com:30548: [Errno 111] Connection refused')>]>